<a href="https://colab.research.google.com/github/alunfes/1m-btc-data/blob/master/RL_Context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
#%cp -rp '/content/drive/My Drive/ta-lib/' ~/
%cd ~
!rm -rf ta-lib*
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!wget https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd '/content/drive/My Drive/ta-lib'
#%cd ta-lib
import os
#os.chdir('/content/drive/My Drive/ta-lib') # Can't use !cd in co-lab
os.chdir('ta-lib') # Can't use !cd in co-lab
#%cd ~/ta-lib
!./configure --prefix=/usr
#! '/content/drive/My Drive/ta-lib/configure' --prefix=/usr
!make
!make install
!pip install Ta-Lib
%cd /content

/root
--2023-01-31 10:19:20--  https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving sourceforge.net (sourceforge.net)... 104.18.10.128, 104.18.11.128, 2606:4700::6812:a80, ...
Connecting to sourceforge.net (sourceforge.net)|104.18.10.128|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/ [following]
--2023-01-31 10:19:20--  https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/
Reusing existing connection to sourceforge.net:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/download [following]
--2023-01-31 10:19:20--  https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/download
Reusing existing connection to sourceforge.net:443.
HTTP request sent, awaiting r

In [4]:
!pip install stable-baselines3
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616822 sha256=4c6b1e25ccbc4d83c28a929e5c4d23979397607147e1311d016eb75525d0917c
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [5]:
%load_ext cython

In [6]:
import numpy as np
import pandas as pd
import gym
import talib as ta
import matplotlib.pyplot as plt
import random
import time
import cython

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, RobustScaler, QuantileTransformer
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torch.optim import Adam
from collections import deque, namedtuple

In [7]:
class MarketData:
    @classmethod
    def initialize(cls, terms):
        cls.terms = terms
        cls.df = pd.DataFrame()
        cls.ts = []
        cls.open = []
        cls.high = []
        cls.low = []
        cls.close = []
        cls.vol = []
        cls.__read_data()
        cls.logopen = list(np.log(cls.open))
        cls.loghigh = list(np.log(cls.high))
        cls.loglow = list(np.log(cls.low))
        cls.logclose = list(np.log(cls.close))
        cls.close_change_ratio = list(np.array(pd.DataFrame(cls.close).pct_change().values.tolist()).reshape(len(cls.close)))
        cls.logclose_change_ratio = list(np.array(pd.DataFrame(cls.logclose).pct_change().values.tolist()).reshape(len(cls.logclose)))
        cls.index_name = []
        cls.index_data = {}
        cls.__calc_index()
    
    @classmethod
    def __read_data(cls):
        cls.df = pd.read_csv('/content/drive/My Drive/BTC-PERP-1mcp.csv')
        cls.ts = list(cls.df['ts'])
        cls.open = list(cls.df['open'])
        cls.high = list(cls.df['high'])
        cls.low = list(cls.df['low'])
        cls.close = list(cls.df['close'])
        cls.vol = list(cls.df['volume'])

    @classmethod
    def __calc_index(cls):
        for term in cls.terms:
            cls.index_name.append('ma_kairi-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_ma_kairi(term)
            cls.index_name.append('rsi-' + str(term))
            cls.index_data[cls.index_name[-1]] =  cls.__calc_rsi(term)
            cls.index_name.append('williams_r-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_williams_R(term)
            #cls.index_name.append('aroon_os-' + str(term))
            #cls.index_data[cls.index_name[-1]] = cls.__calc_aroon_os(term)
            cls.index_name.append('cci-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_cci(term)
            cls.index_name.append('dx-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_dx(term)
            cls.index_name.append('stochastic-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_stochastic(term)
            cls.index_name.append('cmo-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_cmo(term)
            cls.index_name.append('adxr-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_adxr(term)
            #cls.index_name.append('apo-' + str(term))
            #cls.index_data[cls.index_name[-1]] = cls.__calc_apo(term)
            cls.index_name.append('plus_minus_di-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_plus_minus_di(term)
            cls.index_name.append('ppo-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_ppo(term)
            cls.index_name.append('ultosc-' + str(term))
            cls.index_data[cls.index_name[-1]] = cls.__calc_ultosc(term)
        cls.df = pd.DataFrame.from_dict(cls.index_data)

    @classmethod
    def __calc_ma_kairi(cls, term):
        ma = list(ta.MA(np.array(cls.close, dtype='f8'), timeperiod=term))
        return list(map(lambda c, e: (c - e) / e, np.array(cls.close, dtype='f8'), np.array(ma, dtype='f8')))
    
    @classmethod
    def __calc_rsi(cls, term):
        return ta.RSI(np.array(cls.close, dtype='f8'), timeperiod=term)
    
    @classmethod
    def __calc_williams_R(cls, term):
        return list(ta.WILLR(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'),timeperiod=term))
        
    @classmethod
    def __calc_macdfix(cls, term):
        slowperiod = term
        fastperiod= int(term / 2.0)
        signalperiod=int(term / 3.0)
        macd, signal, hist =  ta.MACD(np.array(cls.close, dtype='f8'), np.array(fastperiod, dtype='i8'), np.array(slowperiod, dtype='i8'), np.array(signalperiod, dtype='i8'))
        return macd

    @classmethod
    def __calc_aroon_os(cls, term):
        return list(ta.AROONOSC(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), timeperiod=term))

    @classmethod
    def __calc_cci(cls, term):
        return list(ta.CCI(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term))

    @classmethod
    def __calc_dx(cls, term):
        return list(ta.DX(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term))

    @classmethod
    def __calc_stochastic(cls, term):
        k_term = term
        d_term = int(term / 2.0)
        fastk, fastd = ta.STOCHF(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), fastk_period=k_term, fastd_period=d_term, fastd_matype=0)
        return list(np.array(fastk) / np.array(fastd))


    @classmethod
    def __calc_cmo(cls, term):
        return list(ta.CMO(np.array(cls.close, dtype='f8'), timeperiod=term))

    @classmethod
    def __calc_adxr(cls, term):
        return ta.ADXR(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term)

    @classmethod
    def __calc_apo(cls, term):
        fperiod=term
        speriod=term * 2
        return ta.APO(np.array(cls.close, dtype='f8'), fastperiod=fperiod, slowperiod=speriod, matype=0)

    @classmethod
    def __calc_plus_minus_di(cls, term):
        mdi = ta.MINUS_DI(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term)
        pdi = ta.PLUS_DI(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term)
        return list(np.array(mdi) / np.array(pdi))

    @classmethod
    def __calc_ppo(cls, term):
        fperiod = term
        speriod = term * 2
        return ta.PPO(np.array(cls.close, dtype='f8'), fastperiod=fperiod, slowperiod=speriod, matype=0)

    @classmethod
    def __calc_ultosc(cls, term):
        p2 = int(term / 2.0)
        p1 = int(term / 3.0)
        return ta.ULTOSC(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod1=p1, timeperiod2=p2, timeperiod3=term)

MarketData.initialize([30, 60])

In [ ]:
################################################################################
################################ Positional Encoder #################################
################################################################################
class PositionalEncoder:
    def getPositionEncoding(self, seq_len, dim, n):
        p = np.zeros((seq_len, dim))
        for k in range(seq_len):
            for i in np.arange(int(dim/2)):
                denominator = np.power(n, 2*i/dim)
                p[k, 2*i] = np.sin(k/denominator)
                p[k, 2*i+1] = np.cos(k/denominator)
        return p

In [ ]:
################################################################################
################################ Data Standardization #################################
################################################################################
class DataStandardization:
    def standardScaler(self):
        standardized_data = {}
        sc = StandardScaler()
        for key in list(MarketData.index_data.keys()):
            standardized_data[key] = sc.fit_transform(np.array(MarketData.index_data[key]).reshape(-1,1)).flatten()
        return pd.DataFrame.from_dict(standardized_data)

    def minmaxScalerMarketData(self, df, from_ind, to_ind):
        scaled_data = {}
        ms = MinMaxScaler()
        for col in df.columns:
            scaled_data[col] = ms.fit_transform(np.array(df[col]).reshape(-1,1)).flatten()
        return pd.DataFrame.from_dict(scaled_data)

    def minmaxScalerAction(self, data, from_ind, to_ind):
        scaled_data = {}
        ms = MinMaxScaler((-1, 1))
        return ms.fit_transform(np.array(data[from_ind:to_ind+1]).reshape(-1,1)).flatten()


'''        
ds = DataStandardization()
sdf = ds.standardScaler()
for col in sdf:
    plt.figure()
    plt.title(col)
    plt.plot(sdf[col])
plt.show()
'''

[[        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 ...
 [-0.70549083 -1.36664767 -1.1534931  ...  0.72927052  0.51327914
  -1.14069667]
 [-0.43996238 -0.79591931 -0.55769143 ...  0.41003941  0.4959855
  -0.79141164]
 [-0.42307105 -0.80927864 -0.57631023 ...  0.41003941  0.47818179
  -0.53422881]]


'        \nds = DataStandardization()\nsdf = ds.standardScaler()\nfor col in sdf:\n    plt.figure()\n    plt.title(col)\n    plt.plot(sdf[col])\nplt.show()\n'

In [ ]:
sdf.values

array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [-0.70549083, -1.36664767, -1.1534931 , ...,  0.72927052,
         0.51327914, -1.14069667],
       [-0.43996238, -0.79591931, -0.55769143, ...,  0.41003941,
         0.4959855 , -0.79141164],
       [-0.42307105, -0.80927864, -0.57631023, ...,  0.41003941,
         0.47818179, -0.53422881]])

In [ ]:
################################################################################
################################ Replay Memory #################################
################################################################################
Transition = namedtuple('Transition',
                        (
                            'observation', 'action', 'next_state', 'reward'
                            ))

class ReplayMemory:
    @classmethod
    def initialize(cls, capacity):
        cls.memory = deque([], maxlen=capacity)

    @classmethod
    def push(cls, *args):
        cls.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
################################################################################
################################ Trading Environment #################################
################################################################################
class TradingEnv:
    def __init__(self, from_ind, obs_market_length, obs_market_dim, obs_action_length, reward_period=60, is_done_criterion:float):
        self.from_ind = from_ind
        self.start_ind = random.randint(from_ind, int(len(MarketData.close) * 0.7))
        self.current_ind = self.start_ind
        self.trading_fee = 0.00075
        self.slip_page = 0.00004
        self.is_done_criterion = is_done_criterion #complete trading when asset decreased certain threshold
        self.initial_asset = 10000.0
        self.asset = self.initial_asset
        self.holding_position = 0 #0:none, 1:buy, 2:sell
        self.holding_price = 0
        self.holding_size = 0
        self.realized_pnl= 0
        self.unrealized_pnl = 0
        self.unrealized_pnl_ratio = 0
        self.num_trade = 0
        self.num_win = 0
        self.obs_market_length = obs_market_length
        self.obs_action_length = obs_action_length
        self.reward_period = reward_period
        self.action_space = spaces.Discrete(3) #0:buy, 1:sell, 3:none
        self.action_log = deque([0] * obs_action_length , maxlen = obs_action_length)
        self.position_log = deque([0] * obs_action_length , maxlen = obs_action_length)
        self.pnl_ratio_log = deque([0] * obs_action_length, maxlen = obs_action_length)
        self.reward_log = deque([0] * obs_action_length, maxlen = obs_action_length)
        self.observation_space = {
                'market_data':[],
                'action_data':[],
             'position_data':[],
             'pnl_data':[],
             'reward_data':[]
             }
        self.ds = DataStandardization()
        self.market_data = self.ds.standardScaler()
        self.reset()
    
    def __observe(self):
        from_ind = self.current_ind - self.obs_market_length
        to_ind = from_ind + self.obs_market_length
        obs_marketdata = self.ds.minmaxScalerMarketData(self.market_data, from_ind, to_ind)
        obs_action = self.ds.minmaxScalerAction(self.action_log, len(self.action_log) - self.obs_action_length, len(self.action_log)-1)
        obs_position = self.position_log[-self.obs_action_length:]
        obs_pnl_ratio = self.pnl_ratio_log[-self.obs_action_length:]
        obs_reward = self.reward_log[-self.obs_action_length:]
        return {'market_data':obs_marketdata, 'action_data':obs_action, 'position_data':obs_position, 'pnl_data':obs_pnl_ratio, 'reward_data':obs_reward}

    def reset(self):
        self.start_ind = random.randint(self.from_ind, int(len(MarketData.close) * 0.7))
        self.current_ind = self.start_ind
        self.asset = self.initial_asset
        self.realized_pnl= 0
        self.unrealized_pnl = 0
        self.unrealized_pnl_ratio = 0
        self.holding_position = 0
        self.holding_price = 0
        self.num_trade = 0
        self.num_win = 0
        self.action_log = deque([0] * self.obs_action_length , maxlen = self.obs_action_length)
        self.position_log = deque([0] * self.obs_action_length , maxlen = self.obs_action_length)
        self.pnl_ratio_log = deque([0] * self.obs_action_length, maxlen = self.obs_action_length)
        self.reward_log = deque([0] * self.obs_action_length, maxlen = self.obs_action_length)
    
    '''
    assetにはinitial assetを入れることで毎回同じ金額のentryとする
    '''
    def __calc_entry_size(self, asset, price):
        return round(asset / price, 4)
    
    def __calc_fee(self, entry_price, size):
        return self.trading_fee * entry_price * size


    def step(self, action):
        if action == 1 and self.holding_position == 0:
            self.holding_position = 1
            self.holding_price = MarketData.close[self.current_ind]
            self.holding_size = self.__calc_entry_size(self.initial_asset, self.holding_price)
            self.realized_pnl -= self.__calc_fee(MarketData.close[self.current_ind], self.holding_size)
        elif action == 1 and self.holding_position == 2:
            pnl = self.holding_size * (self.holding_price - MarketData.close[self.current_ind])
            if pnl > 0:
                self.num_win += 1
            self.realized_pnl = pnl
            self.holding_position = 0
            self.holding_price = 0
            self.holding_size = 0
            self.num_trade += 1
        elif action == 2 and self.holding_position == 0:
            self.holding_position = 2
            self.holding_price = MarketData.close[self.current_ind]
            self.holding_size = self.__calc_entry_size(self.initial_asset, self.holding_price)
        elif action == 2 and self.holding_position == 1:
            pnl = self.holding_size * (MarketData.close[self.current_ind] - self.holding_price)
            if pnl > 0:
                self.num_win += 1
            self.realized_pnl = pnl
            self.holding_position = 0
            self.holding_price = 0
            self.holding_size = 0
            self.num_trade += 1
        elif self.holding_position != 0:
            if self.holding_position == 1:
                self.unrealized_pnl = self.holding_size * (MarketData.close[self.current_ind] - self.holding_price)
            elif self.holding_position == 2:
                self.unrealized_pnl = self.holding_size * (self.holding_price - MarketData.close[self.current_ind])
        self.current_ind += 1
        self.is_done_criterion = self.__check_completion()
        if self.is_done_criterion == True:
            return True
        else:
            return False

    '''
    現在のポジションのreward_periodにおける平均pnlの差をrewardとする。
    '''
    def get_reward(self, reward_period, action, entry_price, entry_size):
        if self.holding_position == 0 and action ==0:
            return 0
        elif self.holding_position == 0 and action != 0:
            if action == 1:
                return entry_size * (mean(MarketData.close[self.current_ind : self.current_ind + reward_period]) - entry_price)
            else:
                return entry_size * (entry_price - mean(MarketData.close[self.current_ind : self.current_ind + reward_period]))
        elif self.holding_position == 1 and action == 2:x`




    

    def __check_completion(self):
        if self.current_ind >= (len(MarketData.close) - self.reward_period - 1): #check index
            return True
        elif (self.realized_pnl + self.unrealized_pnl) / self.initial_asset < self.is_done_criterion: #check performance
            return True
        else:
            return False

        


In [ ]:
################################################################################
################################ Trading Environment #################################
################################################################################
'''
market dataに加えて過去の行動とpnl推移を入力に使う。

'''
class TradingEnv2(gym.Env):
    metadata = {'render.modes':['console']}

    def __init__(self, start_ind:int, obs_term:int, obs_market_data_shape, obs_action_pnl_data_shape, is_done_criterion:float):
        super().__init__()
        ds = DataStandardization()
        self.market_data_df = ds.standardScaler()
        self.trading_fee = 0.00075
        self.slip_page = 0.00004
        self.is_done_criterion = is_done_criterion
        self.initial_asset = 10000.0
        self.asset = self.initial_asset.clone()
        self.obs_term = obs_term
        self.num_reset = 0
        self.start_ind = start_ind
        n_actions = 3 #1:buy, -1:sell, 0:no
        self.action_space = spaces.Discrete(n_actions)
        self.observation_space = spaces.Box(low=0, high=1,
                                            shape=(obs_term, market_data_shape), dtype=np.uint8)
        self.reset()

    def __observe(self):
        obs_dict = {}
        ma_kairi_list = []
        for term in MarketData.terms:
            ma_kairi_list.append(self.__minmax_scaler(MarketData.ma_kairi[term][self.i - self.obs_term:self.i][::10]))
        pl_log = self.__minmax_scaler(self.total_pl_log[-self.obs_term:][::10])
        holding_side_log = np.array(self.holding_side_log[-self.obs_term:])#.reshape(-1,1)
        obs_dict = {
            'ma_kairi':np.array(ma_kairi_list).reshape(len(ma_kairi_list), int(self.obs_term/10)),
            'pl_log':pl_log,
            'holding_side_log':np.array(holding_side_log),
            'holding_side':self.holding_side_log[-1],
        }
        return obs_dict
    
    def __minmax_scaler(self, data):
        return self.scaler.fit_transform(np.array(data).reshape(-1,1))
    
    def step(self, action):
        side = 0
        if self.holding_side_log[-1] == 0 and action != 0:
            side = action
            self.entry_price = MarketData.close[self.i]
            self.realized_pl += self.__calc_fee(self.entry_price)
            self.unrealized_pl = 0.0
            self.holding_lot = self.__calc_entry_lot(self.entry_price)
        elif action !=0 and action != self.holding_side_log[-1]:
            side = 0
            self.realized_pl += self.__calc_realized_pl(self.holding_side_log[-1], MarketData.close[self.i])
            self.unrealized_pl = 0.0
            self.holding_lot = 0.0
            self.num_trade += 1
        else:
            side = self.holding_side_log[-1]
            self.unrealized_pl = self.__calc_unrealized_pl(MarketData.close[self.i])
        self.holding_side_log.append(side)
        self.total_pl_log.append(self.realized_pl + self.unrealized_pl + self.initial_asset)
        reward = self.__get_reward()
        done = self.__is_done()
        self.num_steps += 1
        self.i += 1
        return self.__observe(), reward, done, {'total_pl_log':self.total_pl_log, 'num_trade':self.num_trade}

    def __calc_realized_pl(self, side, price):
        if side == 1:
            return self.holding_lot * (price - self.entry_price) - self.__calc_fee(price)
        else:
            return self.holding_lot * (self.entry_price - price) - self.__calc_fee(price)
    
    def __calc_fee(self, price):
        return (self.holding_lot * price * self.trading_fee) + (self.holding_lot * price * self.slip_page)
    
    def __calc_entry_lot(self, entry_price):
        return round(self.asset / entry_price, 6)
    
    def __calc_unrealized_pl(self, price):
        if self.holding_side_log[-1] == 1:
            return round(self.holding_lot * (price - self.entry_price), 6)
        elif self.holding_side_log[-1] == 2:
            return round(self.holding_lot * (self.entry_price - price), 6)
        else:
            return 0.0
        
    def reset(self):
        self.i = self.start_ind# + random.randrange(10000)
        self.num_steps = 0
        self.total_pl_log = [self.initial_asset] * self.obs_term
        self.holding_side_log = [0] * self.obs_term
        self.realized_pl= 0.0
        self.unrealized_pl = 0.0
        self.entry_price = 0.0
        self.holding_lot = 0.0
        self.num_trade = 0
        self.num_reset += 1
        return self.__observe()
    
    def render(self, mode='console'):
        print('steps=', self.num_steps, 'num_trade=', self.num_trade, ', pl=', self.total_pl_log[-1])
    
    def __is_done(self):
        if (self.total_pl_log[-1] - max(self.total_pl_log)) / max(self.total_pl_log) <= self.is_done_criterion:
            print('#Reset=', self.num_reset, 'steps=', self.num_steps, 'pl=', self.total_pl_log[-1], 'num trade=', self.num_trade)
            return True
        else:
            return False
    
    def __get_reward(self):
        if self.total_pl_log[-1] != 0.0:
            return 100.0 * (self.total_pl_log[-1] - self.total_pl_log[-2]) / self.total_pl_log[-2]
        else:
            return 0.0
        
    def close(self):
        pass

In [ ]:
################################################################################
################################ network #################################
################################################################################
class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [ ]:
class TestEnv(gym.Env):
    metadata = {'render.modes':['console']}

    def __init__(self):
        super().__init__()
        n_actions = 3 #1:buy, -1:sell, 0:no
        self.action_space = spaces.Discrete(n_actions)
        self.observation_space= spaces.Dict({'market':spaces.Box(-1,1,(60, 10)), 'actuon':spaces.Box(0,1, (10,)) })

te = TestEnv()
te.observation_space.sample()

In [ ]:
torch.flatten(torch.from_numpy(np.array(list(te.observation_space.sample().values()))))

In [ ]:
%load_ext Cython
import time

0.0


In [8]:
%load_ext Cython

In [ ]:
%%cython
cdef class Account:
    cdef double balance, net_profit

In [ ]:
t = test()
t.primes(1000)

In [46]:
%%cython -a
import numpy as np
import random
cimport numpy as cnp

cdef class AccountEnv:
    cdef public double is_done_pnl_ratio
    cdef public double cash
    cdef public double market_order_fee
    cdef public double limit_order_fee
    cdef public double total_fee
    cdef public int duration
    cdef public int order_distribution_num
    cdef public dict position
    cdef public list buy_orders
    cdef public list sell_orders
    cdef public list order_ids
    cdef public int order_id_last
    cdef public double realized_pnl
    cdef public double unrealized_pnl
    cdef public int num_trade
    cdef public int num_win
    cdef public double win_rate
    cdef public double ptlc_ratio_start
    cdef public double ptlc_ratio_end
    cdef public double order_ratio_start
    cdef public double order_ratio_end
    cdef public int buy_order_var_type
    cdef public int sell_order_var_type
    cdef public double order_stop_loss_ratio
    cdef public double order_take_profit_ratio
    cdef public double pt_price
    cdef public double lc_price

    cdef public double reward
    cdef public dict state
    cdef public int is_done #0:false, 1:true
    
    '''
    ptlc_ratio_start = should be 0.01 - 0.05 (should be largar than limit order fee)
    ptlc_ratio_end = should be larger than ptlc_ratio_start, 
    order_ratio_start = 0.001 - 0.01
    order_ratio_end = 
    '''
    def __cinit__(self, is_done_pnl_ratio:double, market_order_fee:double, limit_order_fee:double, order_distribution_num:int, ptlc_ratio_start:double, ptlc_ratio_end:double, order_ratio_start:double, order_ratio_end:double):
        self.cash = 10000 #always 
        self.is_done_pnl_ratio = is_done_pnl_ratio
        self.market_order_fee = market_order_fee #0.01
        self.limit_order_fee = limit_order_fee #0.006
        self.order_distribution_num = order_distribution_num
        self.ptlc_ratio_start = ptlc_ratio_start
        self.ptlc_ratio_end = ptlc_ratio_end
        self.order_ratio_start = order_ratio_start #0.005
        self.order_ratio_end = order_ratio_end #0.05
        self.reset()


    cpdef tuple reset(self):
        self.realized_pnl = 0
        self.unrealized_pnl = 0
        self.num_trade = 0
        self.num_win = 0
        self.win_rate = 0
        self.duration = 0
        self.total_fee = 0
        self.duration = 0
        self.__initialize_position()
        self.__initialize_order()
        self.reward = self.__calc_reward()
        self.state = {'position':[0,0,0], 'period':0, 'buy_order_start':0, 'buy_order_end':0, 'sell_order_start':0, 'sell_order_end':0, 'pt':0, 'lc':0, 'unrealized_pnl':0}
        self.is_done = 0
        return (self.state, self.reward, self.is_done)

    '''
    取ったアクションが将来の利益にプラスかマイナスかで判断したい。
    order出すことに報酬を出したい。
    order出しておらず何もしていない時はマイナスの報酬を与える。
    '''
    cpdef double __calc_reward(self):
        rew = 0.0
        if self.position['side'] == 0:
            rew = 1.0
        else:
            rew = 0.0
        return rew


    cpdef dict __generate_state(self, list ohlc):
        pside = np.eye(1, M=3, k=1+self.position['side'])
        pperiod = max((self.position['period'] - 1440) / 1440, 1)
        buy_order_start_divergened = 0 if len(self.buy_orders) == 0 else self.buy_orders[0]['price'] / ohlc[3]
        buy_order_end_divergened = 0 if len(self.buy_orders) == 0 else self.buy_orders[-1]['price'] / ohlc[3]
        sell_order_start_divergened = 0 if len(self.sell_orders) == 0 else self.sell_orders[0]['price'] / ohlc[3]
        sell_order_end_divergened = 0 if len(self.sell_orders) == 0 else self.sell_orders[-1]['price'] / ohlc[3]
        buy_var_type = np.eye(1, M=6, k=1+self.buy_order_var_type)
        sell_var_type = np.eye(1, M=6, k=1+self.sell_order_var_type)
        pt_divergence = 0 if self.pt_price == 0 else self.pt_price / ohlc[3]
        lc_divergence = 0 if self.lc_price == 0 else self.lc_price / ohlc[3]
        unrealized_pnl = 0 if self.position['side'] == 0 else self.unrealized_pnl / self.position['size'] * self.position['price']
        return {'position':pside, 'period':pperiod, 'buy_order_start':buy_order_end_divergened, 'buy_order_end':buy_order_end_divergened, 
                'sell_order_start':sell_order_end_divergened, 'sell_order_end':sell_order_end_divergened, 
                'pt':pt_divergence, 'lc':lc_divergence, 'unrealized_pnl':unrealized_pnl}


    cpdef void __initialize_order(self):
        self.buy_orders = [] #{'id', 'side', 'type', 'price', 'size'} side:1 or -1, type:0:market, 1:limit, size:percentage (0-1)
        self.sell_orders = []
        self.buy_order_var_type = -1 #order var type = 0:一様、1:y=x, 2:y=-x, 3:山形, 4:両端山形
        self.sell_order_var_type = -1 #order var type = 0:一様、1:y=x, 2:y=-x, 3:山形, 4:両端山形
        self.order_ids = []
        self.order_id_last = 0
        self.order_stop_loss_ratio = 0
        self.order_take_profit_ratio = 0
        self.pt_price = 0
        self.lc_price = 0

    '''
    Assumed buy/sell order is always in A-Z order and executed from A
    '''
    cpdef void __remove_order(self, int oside):
        if oside == 1:
            del self.buy_orders[0]
            self.buy_order_var_type = -1
        else:
            del self.sell_orders[0]
            self.sell_order_var_type = -1

    cpdef void __initialize_position(self):
        self.position = {'side':0, 'price':0, 'size':0, 'start_duration':0, 'period':0} #side (0:none, 1:buy, -1:sell)
    
    cpdef void __update_position(self, int side, double size, double price, int start_duration, int period):
        self.position = {'side':side, 'size':size, 'price':price, 'start_duration':self.duration, 'period':period}

    '''
    price_start_ratio, price_end_ratio: i.e. start = 0.01, end = 0.01, current price= 10000 then order = [10100, 10120,,,, 10201]
    '''
    cpdef tuple step(self, int action, int order_var_type, double price_start_ratio, double price_end_ratio, double pt_ratio, double lc_ratio, list ohlc):
        #action = 0:nothing, 1:buy, 2:sell, 3:pt, 4:lc, 5:cancel buy, 6:cancel sell, 7:market close
        #Buy: price start ratio,  price end ratio= (0.01 - 0.05)
        #Sell: price start ratio,  price end ratio= (0.01 - 0.05)
        #pt_ratio = (0.01 - 0.05)
        #lc_ratio = (0.005 - 0.05)
        #order var type = 0:一様、1:y=x, 2:y=-x, 3:山形, 4:両端山形
        #

        if self.position['side'] != 0:
            self.position['period'] += 1

        if self.position['side'] == 0 and len(self.buy_orders) == 0 and action == 1:#entry buy limit order
            self.__entry_order(1, 1, 0.1, 0.1, 1, ohlc)
        elif self.position['side'] == 0 and len(self.sell_orders) == 0 and action == 2:#entry sell limit order
            self.__entry_order(1, -1, 0.1, 0.1, 1, ohlc) 
        elif self.position['side'] != 0  and action == 3:#place pt order
            self.__entry_pt_order(pt_ratio)
        elif self.position['side'] != 0 and action == 4: #place lc order
            self.__entry_lc_order(lc_ratio)
        elif len(self.buy_orders) > 0 and action == 5:#cancel buy
            self.__cancel_order(1)
        elif len(self.sell_orders) > 0 and action == 6:#cancel sell
            self.__cancel_order(-1)
        elif self.position['side'] != 0 and action == 7:
            self.__exit_all(ohlc)
        self.__check__execution(ohlc)
        self.__update_cash_pnl(ohlc)
        self.duration += 1
        self.is_done = self.__check_is_done()
        self.reward = self.__calc_reward()
        self.state = self.__generate_state(ohlc)
        return (self.state, self.is_done, self.reward,0)
    
    cpdef int __check_is_done(self):
        if self.is_done_pnl_ratio <= (self.realized_pnl + self.unrealized_pnl - self.total_fee) / self.cash:
            return 0
        else:
            return 1

    cpdef void __update_cash_pnl(self, ohlc):
        if self.position['side'] != 0:
            self.unrealized_pnl = self.cash * self.position['size'] *  (ohlc[3] - self.position['price']) if self.position['side'] == 1 else self.cash * self.position['size'] *  (self.position['price'] - ohlc[3])
        else:
            self.unrealized_pnl = 0


    cpdef tuple action_space_sample(self):
        action_id = random.randint(0,7)
        buy_order_price_start = random.randint(-100, 100) / 100.0
        sell_order_price_start = random.randint(-100, 100) / 100.0
        order_var_type = random.randint(0, 4)
        pt_ratio = random.randint(-100, 100) / 100.0
        lc_ratio = random.randint(-100, 100) / 100.0
        action = (action_id, buy_order_price_start, sell_order_price_start, order_var_type, pt_ratio, lc_ratio)
        return action

    '''
    otype should be always 1 as limit order.
    market order is allowed only for exit_all, and stop loss market order
    '''
    cpdef void __entry_order(self, int otype, int side, double price_start_ratio, double price_end_ratio, int order_var_type, list ohlc):
        order_size_ditribution = self.__generate_order_percentage(order_var_type)
        order_prices = self.__generate_order_prices(price_start_ratio, price_end_ratio, ohlc[0])
        for i in range(len(order_size_ditribution)):
            if side ==1:
                print('buy entry at ', order_prices[i], ', size=', order_size_ditribution[i])
                self.buy_orders.append({'id':self.order_id_last, 'side':side, 'type':otype, 'price':order_prices[i], 'size':order_size_ditribution[i]})
                self.buy_order_var_type = order_var_type
            else:
                print('sell entry at ', order_prices[i], ', size=', order_size_ditribution[i])
                self.sell_orders.append({'id':self.order_id_last, 'side':side, 'type':otype, 'price':order_prices[i], 'size':order_size_ditribution[i]})
                self.sell_order_var_type = order_var_type
            self.order_ids.append(self.order_id_last)
            self.order_id_last += 1
    
    '''
    Assumed output val of NN for pt ratio is -1 ~ 1
    '''
    cpdef void __entry_pt_order(self, double pt_ratio):
        if self.position['side'] != 0:
            self.order_take_profit_ratio = pt_ratio * (self.ptlc_ratio_start -self.ptlc_ratio_end) / 2.0 + (self.ptlc_ratio_start+self.ptlc_ratio_end) / 2.0
            self.pt_price = self.position['price'] * (1.0 + self.order_take_profit_ratio) if self.position['side'] == 1 else self.position['price'] * (1.0 - self.order_take_profit_ratio)
            print('pt entry at ', self.pt_price)

    '''
    Assumed output val of NN for lc ratio is -1 ~ 1
    '''
    cpdef void __entry_lc_order(self, double lc_ratio):
        if self.position['side'] != 0:
            self.order_stop_loss_ratio = lc_ratio * (self.ptlc_ratio_start -self.ptlc_ratio_end) / 2.0 + (self.ptlc_ratio_start+self.ptlc_ratio_end) / 2.0
            self.lc_price = self.position['price'] * (1.0 - self.order_stop_loss_ratio) if self.position['side'] == 1 else self.position['price'] * (1.0 + self.order_stop_loss_ratio)
            print('lc entry at ', self.lc_price)


    cpdef void __exit_all(self, ohlc):
        self.__initialize_order()
        fee = self.__calc_fee(0, self.position['size'], ohlc[0])
        self.total_fee += fee
        realized_pnl= self.__calc_realized_pnl(self.position['size'], ohlc[0])
        if realized_pnl > 0:
            self.num_win +=1
        self.num_trade += 1
        self.realized_pnl += realized_pnl
        self.unrealized_pnl = 0


    cpdef cnp.ndarray __generate_order_percentage(self, int order_var_type):
        order_size_ditribution = np.zeros(1)
        if order_var_type == 0: #uniform distribution
            per = 1.0 / self.order_distribution_num
            order_size_ditribution = np.full(self.order_distribution_num, per) / self.cash
        elif order_var_type == 1: #y=x
            order_size_ditribution = np.linspace(1, self.order_distribution_num+1, self.order_distribution_num)
            order_size_ditribution = (order_size_ditribution / order_size_ditribution.sum()) / self.cash
        elif order_var_type == 2: #y=-x
            order_size_ditribution = np.linspace(self.order_distribution_num+1, 1, self.order_distribution_num)
            order_size_ditribution = (order_size_ditribution / order_size_ditribution.sum()) / self.cash
        elif order_var_type == 3: #/\
            order_size_ditribution = np.concatenate([np.linspace(1, self.order_distribution_num+1, int(self.order_distribution_num * 0.5)), 
                                                     np.linspace(self.order_distribution_num+2, 1, self.order_distribution_num - int(self.order_distribution_num * 0.5))])
            order_size_ditribution = (order_size_ditribution / order_size_ditribution.sum()) / self.cash
        elif order_var_type == 4: #\/
            order_size_ditribution = np.concatenate([np.linspace(self.order_distribution_num+1, 1, int(self.order_distribution_num * 0.5)), 
                                                     np.linspace(2, self.order_distribution_num+1, self.order_distribution_num - int(self.order_distribution_num * 0.5))])
            order_size_ditribution = (order_size_ditribution / order_size_ditribution.sum()) / self.cash
        return order_size_ditribution


    cpdef cnp.ndarray __generate_order_prices(self, double price_start_ratio, double price_end_ratio, double current_price):
        price_start_ratio * (self.order_ratio_start -self.order_ratio_end) / 2.0 + (self.order_ratio_start+self.order_ratio_end) / 2.0
        price_end_ratio * (self.order_ratio_start -self.order_ratio_end) / 2.0 + (self.order_ratio_start+self.order_ratio_end) / 2.0
        s = (1.0 + price_start_ratio) * current_price
        e = (1.0 + price_end_ratio) * s
        print('price start ratio=', price_start_ratio, 'price end ratio=', price_end_ratio, 's=',s, 'e=',e)
        return np.linspace(s, e, self.order_distribution_num)

    
    cpdef void __cancel_order(self, int side):
        if side == 1:
            self.buy_orders = []
        elif side == -1:
            self.sell_orders = []
        else:
            print('AccountEnv: Invalid side in cancel order !')

    '''
    0: no execution, 1:buy order exec, 2:sell order exec, 3:pt exec, 4:lc exec, 5:multiple exec
    '''
    cpdef int __check__execution(self, ohlc):
        check_prices = []
        if ohlc[3] > ohlc[0]: #yosen
            check_prices = [ohlc[2], ohlc[1]]
        else:
            check_prices = [ohlc[1], ohlc[2]]
        for p in check_prices:
            if self.order_take_profit_ratio != 0 and self.position['side'] == 1: #pt lc check for buy position
                if p > self.pt_price:
                    self.__process_pt_order(self.pt_price)
                    return 3
                elif p <= self.lc_price:
                    self.__process_lc_order(self.lc_price)
                    return 4
            elif self.order_take_profit_ratio != 0 and self.position['side'] == -1: #pt lc check for sell position            
                if p < self.pt_price:
                    self.__process_pt_order(self.pt_price)
                    return 3
                elif p >= self.lc_price:
                    self.__process_lc_order(self.lc_price)
                    return 4
        for order in self.buy_orders:
            if p < order['price']:
                print('buy executed: buy=', order['price'], ', price=', p)
                self.__process_execution(order['type'], order['side'], order['id'], order['size'], order['price'])
            else:
                break #orders are in price order (A-Z) so should be executed from A and can exit from loop if current order was not executed
        for order in self.sell_orders:
            if p > order['price']:
                print('sell executed: sell=', order['price'], ', price=', p)
                self.__process_execution(order['type'], order['side'], order['id'], order['size'], order['price'])
                break
        return 0
            
        
    
    cpdef void __process_execution(self, int otype, int oside, int oid, double size, double exec_price):
        fee = self.__calc_fee(otype, size, exec_price)
        self.total_fee += fee
        if self.position['side'] == 0: #new entry
            print('new entry at ', exec_price)
            self.__update_position(oside, size, exec_price, self.duration, 0)
            self.__remove_order(oside)
        elif self.position['side'] != oside: #exit
            realized_pnl = self.__calc_realized_pnl(size, exec_price)
            self.realized_pnl += realized_pnl
            if self.position['size'] > size:
                self.__update_position(self.position['side'], self.position['size'] - size, self.position['price'], self.position['start_duration'], self.position['period'])
                self.__remove_order(oside)
            elif self.position['size'] == size:
                self.__initialize_position()
                self.__initialize_order()
                self.num_trade += 1
                if realized_pnl > 0:
                    self.num_win += 1
            else:
                self.__update_position(oside, size - self.position['size'], exec_price, self.duration, 0)
                self.__remove_order(oside)
                print('AccountEnv: exit and opposite entry is not allowed !')
        else: #additional entry
            updated_price = (self.position['size'] * self.position['price'] + size * exec_price) / (self.position['size'] + size)
            self.__update_position(self.position['side'], self.position['size'] + size, updated_price, self.position['start_duration'], self.position['period'])
            self.pt_price = self.position['price'] * (1.0 + self.order_take_profit_ratio) if self.position['side'] == 1 else self.position['price'] * (1.0 - self.order_take_profit_ratio)
            self.lc_price = self.position['price'] * (1.0 - self.order_stop_loss_ratio) if self.position['side'] == 1 else self.position['price'] * (1.0 + self.order_stop_loss_ratio)
        self.__remove_order(oside)


    cpdef void __process_pt_order(self, double pt_price):
        self.total_fee += self.__calc_fee(1, self.position['size'], pt_price)
        realized_pnl = self.__calc_realized_pnl(self.position['size'], pt_price)
        self.realized_pnl += realized_pnl
        self.num_trade += 1
        if realized_pnl > 0:
            self.num_win += 1
        self.__initialize_position()
        self.__initialize_order()

    cpdef void __process_lc_order(self, double lc_price):
        self.total_fee += self.__calc_fee(0, self.position['size'], lc_price)
        realized_pnl = self.__calc_realized_pnl(self.position['size'], lc_price)
        self.realized_pnl += realized_pnl
        self.num_trade += 1
        self.__initialize_position()
        self.__initialize_order()
    
    cpdef double __calc_fee(self, int otype, double size, double exec_price):
        if otype == 0:
            return self.market_order_fee * size * exec_price * self.cash
        elif otype == 1:
            return self.limit_order_fee * size * exec_price * self.cash
    
    cpdef double __calc_realized_pnl(self, double size, double exec_price):
        if self.position['side'] == 1:
            return  (exec_price - self.position['price']) * size * self.cash
        elif self.position['side'] == -1:
            return (self.position['price'] - exec_price) * size * self.cash
        else:
            print('AcountEnv: realized pnl should not be called when no position !')
            return 0




In [47]:
is_done_pnl_ratio = -0.1
market_order_fee = 0.01
limit_order_fee = 0.06
order_distribution_num = 10
ptlc_ratio_start = 0.01
ptlc_ratio_end = 0.05
order_ratio_start = 0.005
order_ratio_end = 0.05

ac = AccountEnv(is_done_pnl_ratio, market_order_fee, limit_order_fee, order_distribution_num, ptlc_ratio_start, ptlc_ratio_end, order_ratio_start, order_ratio_end)
ac.reset()
start_ind = 1000000
for i in range(30):
    ohlc = [MarketData.open[start_ind + i], MarketData.high[start_ind + i],MarketData.low[start_ind + i],MarketData.close[start_ind + i]]
    action = ac.action_space_sample()
    print(action)
    res = ac.step(action[0], action[1], action[2], action[3], action[4], action[5], ohlc)
    print('i=',i, res)

(2, 0.55, -0.8, 1, -0.95, 0.42)
price start ratio= 0.1 price end ratio= 0.1 s= 36033.8 e= 39637.18000000001
sell entry at  36033.8 , size= 1.6666666666666667e-06
sell entry at  36434.17555555556 , size= 3.5185185185185187e-06
sell entry at  36834.55111111111 , size= 5.37037037037037e-06
sell entry at  37234.92666666667 , size= 7.222222222222223e-06
sell entry at  37635.30222222223 , size= 9.074074074074075e-06
sell entry at  38035.67777777778 , size= 1.0925925925925925e-05
sell entry at  38436.05333333334 , size= 1.2777777777777779e-05
sell entry at  38836.4288888889 , size= 1.4629629629629631e-05
sell entry at  39236.80444444445 , size= 1.6481481481481482e-05
sell entry at  39637.18000000001 , size= 1.8333333333333333e-05
i= 0 ({'position': array([[0., 1., 0.]]), 'period': 1, 'buy_order_start': 0, 'buy_order_end': 0, 'sell_order_start': 1.2092248085664605, 'sell_order_end': 1.2092248085664605, 'pt': 0, 'lc': 0, 'unrealized_pnl': 0}, 0, 1.0, 0)
(4, -0.25, -0.17, 0, -0.74, 0.54)
i= 1 ({

In [38]:
MarketData.high[1000000]

33278.0

In [50]:
current_price = 33000

0.8 * (order_ratio_start -order_ratio_end) / 2.0 + (order_ratio_start+order_ratio_end) / 2.0
-0.6 * (order_ratio_start -order_ratio_end) / 2.0 + (order_ratio_start+order_ratio_end) / 2.0
s = (1.0 + 0.01) * current_price
e = (1.0 + 0.01) * s
np.linspace(s, e, 10)

array([33330.        , 33367.03333333, 33404.06666667, 33441.1       ,
       33478.13333333, 33515.16666667, 33552.2       , 33589.23333333,
       33626.26666667, 33663.3       ])

In [49]:
ovar

NameError: ignored

In [23]:
import time
orders = []
orders.append({'id':1, 'price':100})
orders.append({'id':2, 'price':101})
orders.append({'id':3, 'price':102})
orders.append({'id':4, 'price':103})
st = time.time()
ind = next((index for (index, d) in enumerate(orders) if d['id'] == 3), None)
print(ind, 't=', time.time()-st)

2 t= 0.00017523765563964844


In [12]:
random.randint(0,4)

3